In [ ]:
import polars as pl
import os
import sys
import json
from dotenv import load_dotenv

sys.path.append(os.path.dirname(os.getcwd()))

from libraries.client_stashapp import StashAppClient, get_stashapp_client

load_dotenv()

stash_client = StashAppClient()
stash_raw_client = get_stashapp_client()

In [ ]:
stashapp_studio_id = 1336
studio_scenes = stash_client.find_scenes_by_studio([stashapp_studio_id])

In [ ]:
def read_json_sidecar(primary_file_path: str):
    primary_file_path_without_suffix = os.path.splitext(primary_file_path)[0]
    info_json_path = primary_file_path_without_suffix + ".info.json"
    print(info_json_path)

    if os.path.exists(info_json_path):
        with open(info_json_path, "r") as f:
            info_json = json.load(f)
            del info_json["formats"]
            # Log the JSON data
            print(f"Read JSON data: {info_json}")
            # Convert all numeric values to float
            info_json = {k: float(v) if isinstance(v, (int, float)) else v for k, v in info_json.items()}
            return info_json
    return {}

studio_scenes = studio_scenes.with_columns(
    pl.col("stashapp_primary_file_path").map_elements(read_json_sidecar, return_dtype=pl.Struct).alias("info_json")
).unnest("info_json")

In [ ]:
offset = 0
batch_size = 100

In [ ]:
current_scenes = studio_scenes.slice(offset, batch_size)
offset += batch_size
current_scenes.select("stashapp_id", "id", "title", "upload_date", "thumbnail", "webpage_url")

In [ ]:
import requests
import base64
from datetime import datetime

for row in current_scenes.iter_rows(named=True):
    thumbnail_url = row["thumbnail"]
 
    # Download thumbnail and convert to base64
    thumbnail_response = requests.get(thumbnail_url)
    thumbnail_base64 = base64.b64encode(thumbnail_response.content).decode("utf-8")
    cover_image_base64 = f"data:image/jpeg;base64,{thumbnail_base64}"

    stash_raw_client.update_scene({
        "id": row["stashapp_id"],
        "code": row["id"],
        "title": row["title"],
        "date": datetime.strptime(row["upload_date"], "%Y%m%d").strftime("%Y-%m-%d"),
        "urls": row["webpage_url"],
        "cover_image": cover_image_base64,
    })